In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import time
import datetime

# I. First steps (Discovering API with various functions)


## Using Riot API key

In [2]:
#Complete with your own API key 
api_key = 'RGAPI-xxxx'
watcher = LolWatcher(api_key)
my_region = 'euw1'
#["na1", "euw1", "eun1", "kr", "br1", "jp1", "ru", "oc1", "tr1", "la1", "la2"]
my_summoner='MamaYumYum'

#### Profile information

In [3]:
#Load a profile  
my_profile = watcher.summoner.by_name(my_region,my_summoner)

In [4]:
print(my_profile)

{'id': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0', 'accountId': 'RiVjJkjVb_T-e_hgpmXYL6oLTO6G8uGjre8oOQM7-sdaSQ', 'puuid': 'La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw', 'name': 'MamaYumYum', 'profileIconId': 5622, 'revisionDate': 1680818073000, 'summonerLevel': 525}


In [5]:
#Load ranked stats
my_ranked_stats = watcher.league.by_summoner(my_region,my_profile['id'])
print(my_ranked_stats)

[{'leagueId': '50a6cc74-60c4-4896-aa2a-54c73299f7c7', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'PLATINUM', 'rank': 'IV', 'summonerId': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0', 'summonerName': 'MamaYumYum', 'leaguePoints': 47, 'wins': 102, 'losses': 99, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


## Match information using JSON

In [6]:
#This function returns a list of the last played game with the game's ids
def get_matches_list(my_region,my_summoner,nbMatches):
    profile=watcher.summoner.by_name(my_region,my_summoner)
    matches_list=watcher.match.matchlist_by_puuid(my_region,profile['puuid'],count=nbMatches)
   
    return matches_list
    
#The function matchlist_by_puuid has a limit of 100 games

In [7]:
nbMatches=20
matches_list=get_matches_list(my_region,my_summoner,nbMatches)

In [8]:
matches_list

['EUW1_6349558955',
 'EUW1_6349503397',
 'EUW1_6347767990',
 'EUW1_6346829954',
 'EUW1_6346406135',
 'EUW1_6346377480',
 'EUW1_6345492861',
 'EUW1_6344920335',
 'EUW1_6344828393',
 'EUW1_6344780310',
 'EUW1_6340799573',
 'EUW1_6340749762',
 'EUW1_6335975261',
 'EUW1_6335029842',
 'EUW1_6334965764',
 'EUW1_6333590234',
 'EUW1_6333492894',
 'EUW1_6328709921',
 'EUW1_6328662160',
 'EUW1_6328212297']

In [85]:
matches_list[0]

'EUW1_6349558955'

In [9]:
#The match.by_id function returns a JSON containing all the information about a game
def get_match(my_region,match_id):
    match=watcher.match.by_id(my_region,match_id)
    return match

In [10]:
#We work with the first match of the list
match=get_match(my_region,matches_list[0])
match['metadata']['matchId']

'EUW1_6349558955'

#### Getting list of summoners's puuid in the match using metadata item

In [11]:
#List of all the game's participants with puuids
match['metadata']['participants']

['La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw',
 'XHO77Tu6kS8doG9PTTKpwE2qabXAl9U7zVz5WeOQ-vSDboIb1TueOw2FbbhKOMhnZIdxphfqm5vZaA',
 'Bk8A3yDI4mUKGgXvAUz8gnFXTbFHjezYlSap4E6i_eYCI96b0h5XmZdaM6L-8s3l76MizQ60SZIy4A',
 'dZvlG4d9pChmlZW6aLSdnYe65sC_Nn5D4YUA0VXTwMkhZrhNC1-gzHfo8ri66SpEHmj4tNbcoD3Ogw',
 'RGqYvPKd2QdADVkFQ8mawBREzipIRl1RFThTpwLUOzv6xyumTdWrW1-Z54r27nuQomIkx17R1baNrQ',
 '9MnJYOYJEpv35p9bVZFanWmGq9GRRiysWQllaPS1Ai3mYyVAJj-YELkl9tm3q_D1cYZ8US8_qDhGNA',
 'O26Iv7GwOaD1geGVDpPjhPb8sqlmq2hwQPfbAo_6E7HyimI6yaS3Za3t4OeRj558InVBD7KYHZt71A',
 'CdqVMgkKa_TL8VdnBw39tStj6Tt8YWoTxoUvM7RyAyQEUZ_jyx4b72lRlp8-xpMpP6kfxDVe7YVqAA',
 'XIotV0X8NoWbqHmZMjOzYnk5rfmvehZz2LJzBmkA_107DmG4i6826u2SEJ9zbhLATzO3G7PAjvjMJg',
 'RMeOALi79F21XWbSxNvnUmtS535WIGMhDbIrTkdDfecpTo7aho823w8kfo5puf0kN07kh4B_xRmT8g']

In [12]:
#get your index in a game, will be usefull later
puuid=my_profile['puuid']
participants = match['metadata']['participants']
# Now, find where in the data our players puuid is found
player_index = participants.index(puuid)
player_index

0

In [13]:
summoner=watcher.summoner.by_puuid(my_region,match['metadata']['participants'][2])
summoner

{'id': '2yWfVNJBjfnX7cP44HMRIbk-y4ERMgyZR8OaH4bitgs5BhM',
 'accountId': 'JhYLPxAvbDjC22cJMKdlGWI1rOHBtuzfKjAAyGvL1dBzLg',
 'puuid': 'Bk8A3yDI4mUKGgXvAUz8gnFXTbFHjezYlSap4E6i_eYCI96b0h5XmZdaM6L-8s3l76MizQ60SZIy4A',
 'name': 'H4riis',
 'profileIconId': 1296,
 'revisionDate': 1680818073000,
 'summonerLevel': 165}

#### Using info

In [14]:
match['info']['participants'][0]['summonerName']

'MamaYumYum'

In [15]:
summoner_name=watcher.summoner.by_puuid(my_region,match['info']['participants'][0]['puuid'])
summoner_name

{'id': 'tfQ1hZABPAxKvX6rkgpqWE7UszeqS8sq7lieyKopKUMu3Z0',
 'accountId': 'RiVjJkjVb_T-e_hgpmXYL6oLTO6G8uGjre8oOQM7-sdaSQ',
 'puuid': 'La6KBkKxEZGOS-jBVIqKsS8PzTvxz7BP_k_AIeK-EkbUrCgHgGy3UW1XGX4h6qN4S7mnIqwIzCKfYw',
 'name': 'MamaYumYum',
 'profileIconId': 5622,
 'revisionDate': 1680818073000,
 'summonerLevel': 525}

#### List of game's participants

In [16]:
def get_list_participants(my_region,match_id):
    match=get_match(my_region,match_id)
    list_participants=[]
    for element in match['metadata']['participants']:
        profile=watcher.summoner.by_puuid(my_region,element)
        list_participants.append(profile['name'])
    return list_participants

In [17]:
list_participants=get_list_participants(my_region,matches_list[0])
list_participants

['MamaYumYum',
 'DreamAboutMe',
 'H4riis',
 'Madnate',
 'A FullStack Dev',
 'Bilbo Lagging',
 'Sam Gadji',
 'Denis Brossard',
 'Conan42O',
 'Pr3YsAK']

#### Conversion unix timestamp

In [18]:
# This function will convert unix time into a date, it can be used for further works
# for ex: Game per day,games played during a certain period etc...
def get_date(unix_time):
    date_time = datetime.datetime.fromtimestamp(unix_time/1000)
    date=date_time.strftime('%Y-%m-%d %H:%M:%S')
    return date

In [19]:

unix_time = match['info']['gameCreation']
get_date(unix_time)

'2023-04-06 23:33:37'

## Game Details  


In [20]:
# This function gives information about the last game using matches_list
#queueId is an integer that gives us the game mode
#400 : Draft game  
#420 : Ranked game  
#440 : Ranked flex  
#450 : ARAM

def game_results_resume(matches_list):
    game_results_df=[]
    for match in matches_list:
        match_json=get_match(my_region,match)
        
        #We get the player_index to get to match_json['info']['participants'][player_index]['win']
        #We can't use match_json['info']['teams']['win'] as we are not appearing at the same index in each game depending
        # on blue side and red side.
        participants = match_json['metadata']['participants']
        player_index = participants.index(puuid)
        
        info_row={}
        info_row['gameId']=match_json['metadata']['matchId']
        info_row['Type']=match_json['info']['gameMode']
        info_row['queueId']=match_json['info']['queueId']
        info_row['Win']=match_json['info']['participants'][player_index]['win']
        info_row['Game Duration']=match_json['info']['gameDuration']/60
        info_row['Date']=get_date(match_json['info']['gameCreation'])
        game_results_df.append(info_row)
    df=pd.DataFrame(game_results_df)
    return df
       
            
    

In [21]:
game_results=game_results_resume(matches_list)
game_results

,gameId,Type,queueId,Win,Game Duration,Date
0,EUW1_6349558955,ARAM,450,True,20.500000,2023-04-06 23:33:37
1,EUW1_6349503397,ARAM,450,False,23.983333,2023-04-06 23:06:15
2,EUW1_6347767990,CLASSIC,400,True,28.600000,2023-04-05 19:31:09
3,EUW1_6346829954,ARAM,450,False,22.266667,2023-04-04 23:12:51
4,EUW1_6346406135,ARAM,450,True,18.950000,2023-04-04 19:30:31
5,EUW1_6346377480,CLASSIC,400,False,15.100000,2023-04-04 19:11:14
6,EUW1_6345492861,ARAM,450,False,20.300000,2023-04-03 23:50:23
7,EUW1_6344920335,ARAM,450,True,19.766667,2023-04-03 18:55:48
8,EUW1_6344828393,CLASSIC,420,True,26.083333,2023-04-03 18:02:53
9,EUW1_6344780310,CLASSIC,420,False,24.866667,2023-04-03 17:29:46


In [22]:
#We build a function that gives a game's summary similar to the ones we can get on opgg or leagueofgraph.
def game_summary(my_region,match_id):
    game_summary_df=[]
    match=get_match(my_region,match_id)
    game_info=match['info']
    
    for row in game_info['participants']:
        participants_row={}
        participants_row['summonerName']=row['summonerName']
        #There is no role or lane in ARAM mode
        if row['individualPosition']=="Invalid":
            participants_row['lane']='ARAM'
        else:  
            participants_row['lane']=row['individualPosition']
        participants_row['champion']=row['championName']
        participants_row['kills']=row['kills']        
        participants_row['deaths']=row['deaths']
        participants_row['assists']=row['assists']
        if row['deaths']==0:
            participants_row['kda']=row['kills']+row['assists']
        else:
            participants_row['kda']=round((row['kills']+row['assists'])/row['deaths']
                                       ,2)
        participants_row['total damages']=row['totalDamageDealtToChampions']
        participants_row['win']=row['win']
        game_summary_df.append(participants_row)
    df=pd.DataFrame(game_summary_df)
    return df
    


In [23]:
matches_list[0]

'EUW1_6349558955'

In [24]:
# I used a game that wasn't an aram
game_summary=game_summary(my_region, matches_list[0])

game_summary


,summonerName,lane,champion,kills,deaths,assists,kda,total damages,win
0,MamaYumYum,ARAM,Vex,10,8,27,4.62,43533,True
1,DreamAboutMe,ARAM,Pyke,11,8,19,3.75,14531,True
2,H4riis,ARAM,Lucian,16,7,15,4.43,29506,True
3,Madnate,ARAM,Vi,10,12,19,2.42,27564,True
4,A FullStack Dev,ARAM,Udyr,4,8,38,5.25,18946,True
5,Bilbo Lagging,ARAM,Jayce,8,13,17,1.92,22914,False
6,Sam Gadji,ARAM,Varus,11,8,17,3.50,26898,False
7,Denis Brossard,ARAM,Urgot,4,13,20,1.85,15471,False
8,Conan42O,ARAM,Caitlyn,9,11,20,2.64,23272,False
9,Pr3YsAK,ARAM,Nidalee,9,6,17,4.33,15302,False


## Champions details

In [25]:
#This function returns a list of the last champions played in the last nbMatches matches
def last_champ_played(my_region, my_summoner,nbMatches):
    last_champ_played=[]
    match_list=get_matches_list(my_region,my_summoner,nbMatches)
    for match in match_list:
        match_played=watcher.match.by_id(my_region,match)
        for participants in match_played['info']['participants']:
            if participants['summonerName']==my_summoner:
                last_champ_played.append(participants['championName'])
                break
    return last_champ_played

In [26]:
last_champ_played=last_champ_played(my_region,my_summoner,nbMatches)
last_champ_played

['Vex',
 'Kennen',
 'Cassiopeia',
 'Singed',
 'Kassadin',
 'Cassiopeia',
 'Vi',
 'AurelionSol',
 'Zed',
 'Yone',
 'Chogath',
 'Zed',
 'Gragas',
 'Neeko',
 'Sylas',
 'Yasuo',
 'Jayce',
 'Shyvana',
 'Fiora',
 'Taric']

In [102]:
#We can do the same function using matches_list only

def list_champ_I_played(matches_list):
    champ_list=[]
    for match in matches_list:
        match_played=watcher.match.by_id(my_region,match)
        for participants in match_played['info']['participants']:
            if participants['summonerName']==my_summoner:
                champ_list.append(participants['championName'])
                break
    return champ_list
    

In [103]:
champ_played=list_champ_I_played(matches_list)
champ_played

['Vex',
 'Kennen',
 'Cassiopeia',
 'Singed',
 'Kassadin',
 'Cassiopeia',
 'Vi',
 'AurelionSol',
 'Zed',
 'Yone',
 'Chogath',
 'Zed',
 'Gragas',
 'Neeko',
 'Sylas',
 'Yasuo',
 'Jayce',
 'Shyvana',
 'Fiora',
 'Taric']

In [130]:
def all_champ_played(matches_list):
    champ_list=[]
    for match in matches_list:
        match_played=watcher.match.by_id(my_region,match)
        for participants in match_played['info']['participants']:
            champ_list.append(participants['championName'])
    return champ_list
    

In [131]:
all_champ=all_champ_played(matches_list)

In [132]:
len(all_champ)

200

In [133]:
#Using Counter to show champs played
#Counter creates a dict
from collections import Counter
z=Counter(all_champ).most_common(15)
z

[('Milio', 5),
 ('Zed', 5),
 ('Vi', 4),
 ('Jayce', 4),
 ('Caitlyn', 4),
 ('Seraphine', 4),
 ('Kaisa', 4),
 ('Annie', 4),
 ('Senna', 4),
 ('Pyke', 3),
 ('Urgot', 3),
 ('Corki', 3),
 ('Nautilus', 3),
 ('MissFortune', 3),
 ('Jhin', 3)]

In [32]:
#Using pd.Series to show champs played
series=pd.Series(all_champ).value_counts()
series


Milio         5
Zed           5
Caitlyn       4
Senna         4
Vi            4
             ..
Viktor        1
MonkeyKing    1
Gwen          1
Talon         1
Taric         1
Length: 109, dtype: int64

## Filtering the games




In [33]:
#We only want ranked or ranked flex games. We get rid of the other games with the attribute 'queueId'.  

def ranked_list(matches_list):   
    ranked_list=[]
    for games in matches_list:
        match_played=watcher.match.by_id(my_region,games)
        if match_played['info']['queueId']== 420 or match_played['info']['queueId']== 440  :
            ranked_list.append(match_played['metadata']['matchId'])
    return ranked_list
            
    

In [34]:
#Only played 3 rankeds in my last 20 games
ranked_list=ranked_list(matches_list)
ranked_list

['EUW1_6344828393', 'EUW1_6344780310', 'EUW1_6333492894']

In [35]:
#Let's see what champs were played on ranked games
champ_ranked=list_champ_played(ranked_list)
champ_ranked

['Zed', 'Yone', 'Jayce']

# II. EUW Challengers

### Loading the CSV  


In [136]:
#We load the csv containing the first 100 challengers of EUW. See tutorial on webscrapping
euw_100 = pd.read_csv('euw_100_challengers')

In [37]:
euw_100.head(10)

,Summoner name,LP,Winrate
0,Razørk Activoo,"1,756 LP",57%
1,ViV NEXT ADKING,"1,745 LP",58%
2,Thumbs Down,"1,647 LP",57%
3,DzuUwU,"1,639 LP",60%
4,I Love Uber Eats,"1,624 LP",54%
5,Noah7,"1,622 LP",59%
6,S P L A S H 777,"1,622 LP",54%
7,4321455213123,"1,604 LP",62%
8,the inescapable,"1,589 LP",60%
9,Odysseus131,"1,557 LP",56%


In [114]:
#We get a list of the summoner's name 
sum_name_euw = euw_100['Summoner name'].tolist()

In [142]:
sum_name_euw[:20]

['Razørk Activoo',
 'ViV NEXT ADKING',
 'Thumbs Down',
 'DzuUwU',
 'I Love Uber Eats',
 'Noah7',
 'S P L A S H 777',
 '4321455213123',
 'the inescapable',
 'Odysseus131',
 'Beanovich',
 'bigboynumbers',
 'supaaaaaaaaaaaaa',
 'Stend',
 'Twitch Sinerias',
 'BDS xMatty',
 'school phobia',
 'PORTUGUESE GALA',
 'Road2Narnia',
 'TakeSet sama']

In [119]:
# We create a list of the 10 last games of each the 20 challengers so we get list of 200 matchs ID
list_game=[]
for name in sum_name_euw[:20]:
    
    l=get_matches_list(my_region,name,10)
    list_game+=l
    

In [143]:
#There is a possibility in all the games that some players were in the same game
#meaning we can get duplicates in the list, we loop inside the list_game to get rid of those games
#Results show that 21 elements of the list were deleted
print(len(list_game))
unique_games=[]
for l in list_game:
        if l not in unique_games:
            unique_games.append(l)

print(len(unique_games))

200
179


In [134]:
#We have 179 unique games, let's see the champions played
champions=all_champ_played(unique_games)

In [149]:
print( "In these 179 games, ", len(counter), " out of 163 different champions were played" )
counter=Counter(champions)
sorted_list=counter.most_common(25)
sorted_list

In these 179 games,  159  out of 163 different champions were played


[('Jinx', 61),
 ('Thresh', 48),
 ('Zeri', 42),
 ('Gragas', 41),
 ('LeeSin', 39),
 ('Rakan', 38),
 ('Xayah', 32),
 ('Senna', 29),
 ('Aphelios', 27),
 ('Jayce', 26),
 ('Sylas', 26),
 ('Viego', 25),
 ('Riven', 24),
 ('Nautilus', 24),
 ('Lulu', 24),
 ('Vi', 22),
 ('Tristana', 22),
 ('Kaisa', 22),
 ('Annie', 22),
 ('Graves', 21),
 ('Irelia', 21),
 ('JarvanIV', 21),
 ('Olaf', 21),
 ('Pyke', 20),
 ('Malphite', 20)]

In [155]:
data=pd.DataFrame(sorted_list, columns=['Champion Name', 'Game appearance'])
data 

,Champion Name,Game appearance
0,Jinx,61
1,Thresh,48
2,Zeri,42
3,Gragas,41
4,LeeSin,39
5,Rakan,38
6,Xayah,32
7,Senna,29
8,Aphelios,27
9,Jayce,26
